In [28]:
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

In [29]:
# Путь к отчетам с сессиями
PATH = r"C:\Users\UmudovBM\Desktop\Час пик"

In [30]:
data_bus = pd.DataFrame(columns=['id_session', 'bus_route', 'bus_route_duration', 'count'])
data_auto = pd.DataFrame(columns=['id_session', 'auto_route', 'auto_route_duration', 'count'])

In [31]:
for id_task in os.listdir(PATH):
    k = 0
    for id_session in os.listdir(os.path.join(PATH, id_task)):
        file_name = os.listdir(os.path.join(PATH, id_task, id_session))
        df = pd.read_excel(os.path.join(PATH, id_task, id_session, *file_name))
        for index, rows in df.iterrows():
            for i in range(1, 6):
                if rows[0] == f'Длительность маршрута {i} на городском транспорте':
                    data_bus.loc[k+i,'bus_route'] = rows[0]
                    data_bus.loc[k+i, 'bus_route_duration'] = rows[1]
                    data_bus.loc[k+i, 'id_session'] = df.loc[0, 'Unnamed: 1']
                    data_bus.loc[k+i, 'count'] = i
            for i in range(1, 6):        
                if rows[0] == f'Длительность маршрута {i} на автомобиле':
                    data_auto.loc[k+i, 'auto_route'] = rows[0]
                    data_auto.loc[k+i, 'auto_route_duration'] = rows[1]
                    data_auto.loc[k+i, 'id_session'] = df.loc[0, 'Unnamed: 1']
                    data_auto.loc[k+i, 'count'] = i
        k += i

In [32]:
data = pd.merge(data_bus,data_auto, on=['id_session','count'])

In [33]:
data['bus_route_duration'] = pd.to_datetime(data['bus_route_duration'], format = '%H:%M', errors = 'coerce')
data['auto_route_duration'] = pd.to_datetime(data['auto_route_duration'], format = '%H:%M', errors = 'coerce')

In [34]:
data['is_twenty'] = data['bus_route_duration'] > '1900-01-01 00:20:00'

In [35]:
def set_session_status(data):
    if data == False:
        return 'TaskNotCompleted'
    return 'TaskCompleted'
def set_session_reason(data):
    if data == False:
        return 'Длительность маршрута на городском транспорте менее 20 минут'   

In [36]:
data['session_status'] = data['is_twenty'].apply(set_session_status)
data['session_reason'] = data['is_twenty'].apply(set_session_reason)

In [37]:
data['difference'] = (data['bus_route_duration'] - data['auto_route_duration']).dt.total_seconds()/60

In [38]:
def set_fast(DATA):
    if DATA['bus_route_duration'] < DATA['auto_route_duration']:
        return f'быстрее на городском транспорте'
    elif DATA['bus_route_duration'] > DATA['auto_route_duration']:
        return f'быстрее на автомобиле'
    else:
         return f'длительность одинаковая' 

In [39]:
data['fast'] = data.apply(set_fast, axis=1)

In [40]:
data['count'] = data['count'].astype(str)

In [41]:
data['fast'] = 'На маршруте' +' '+ data['count'] +' '+ data['fast']

In [42]:
data.loc[data['bus_route_duration'].isna(), 'session_reason'] = 'Некерректно указана длительность маршрута'
data.loc[data['auto_route_duration'].isna(), 'session_reason'] = 'Некерректно указана длительность маршрута'

In [43]:
not_succesfull = data[data['session_status'] == 'TaskNotCompleted']
not_succesfull = not_succesfull[['id_session', 'session_status', 'session_reason']]

In [44]:
update_not_succesfull = not_succesfull.drop_duplicates()

In [45]:
good_data = data.query('id_session not in @update_not_succesfull.id_session')

In [49]:
successfull = good_data[['id_session', 'session_status']]

In [50]:
update_successfull = successfull.drop_duplicates()

In [52]:
update_session = update_not_succesfull.append(update_successfull)

In [54]:
update_not_succesfull.columns = ['id сессии', 'Проверка длительности маршрута', 'Статус']

In [55]:
update_successfull = update_successfull[['id_session', 'session_status']]
update_successfull.columns = ['id сессии', 'Статус']

In [57]:
report = good_data[['id_session', 'fast', 'difference']]
report['difference'] = abs(report['difference'])
report.columns = ['id сессии', 'Сравнение между длительностью на городском транспорте и автомобиле', 'Разница во времени (минут)']

In [58]:
with pd.ExcelWriter(r"C:\Users\UmudovBM\Desktop\Маршруты 02.02.xlsx") as writer:
    update_not_succesfull.to_excel(writer, sheet_name="Отклоненные", index=False)
    update_successfull.to_excel(writer, sheet_name="Успешные", index=False)
    report.to_excel(writer, sheet_name="Отчет", index=False)

In [59]:
update_session.to_excel(r"C:\Users\UmudovBM\Desktop\Обновление сессий ЧАС ПИК 02.02.xlsx", index=False, header=None)